In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write





def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

## Kinyarwanda

In [2]:
# common_voice_rw_21176283.wav
text = '''Aba basore bose uyu mukobwa avugwaho gukundana na bo ngo ibyabo ni ubucuti busanzwe.'''


# DUMMY3/common_voice_rw_21072591.wav|

text2 = '''Muri Nyagatare ho bafunguye ku mugaragaro ikigo nderabuzima cya Gakagati bateye inkunga.'''

In [3]:
hps = utils.get_hparams_from_file("configs/rw_kin.json")

In [4]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("/home/navneeth/EgoPro/dnn/vits_kinyarwanda/models/G_229000.pth", net_g, None)

/home/navneeth/anaconda3/envs/vits/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


INFO:root:Loaded checkpoint '/home/navneeth/EgoPro/dnn/vits_kinyarwanda/models/G_229000.pth' (iteration 243)


In [5]:
l = [text, text2]
directory = "kinyarwanda"
os.makedirs(directory,exist_ok=True)
for i in range(len(l)):
    stn_tst = get_text(l[i], hps)
    with torch.no_grad():
        x_tst = stn_tst.cuda().unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
        audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
    ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))
    write(f"{directory}/old_d{i}.mp3",rate=hps.data.sampling_rate,data=audio)

['a', 'b', 'a']
['b', 'a', 's', 'o', 'r', 'e']
['b', 'o', 's', 'e']
['u', 'y', 'u']
['m', 'u', 'k', 'o', 'bw', 'a']
['a', 'v', 'u', 'g', 'w', 'a', 'h', 'o']
['g', 'u', 'k', 'u', 'n', 'd', 'a', 'n', 'a']
['n', 'a']
['b', 'o']
['n', 'g', 'o']
['i', 'by', 'a', 'b', 'o']
['n', 'i']
['u', 'b', 'u', 'c', 'u', 't', 'i']
['b', 'u', 's', 'a', 'n', 'zw', 'e']


UnicodeDecodeError: 'ascii' codec can't decode byte 0xcb in position 17: ordinal not in range(128)